# Value Discovery State Agent — Two-Phase Exploratory Analysis

This notebook visualizes the outputs of the two-phase Value Discovery State Agent pipeline.
It reads from the generated CSV artifacts in `output/` and does **no new computation** —
all data was produced by the deterministic pipeline scripts.

**Architecture:**
- **Pre-IB (6:30-7:25am PT)**: Discovery vs prior session VAH/VAL
- **Post-IB (7:30am-12:55pm PT)**: Discovery vs IBH/IBL, extensions normalized by IB range

**Sections:**
1. Overview: Pre-IB vs Post-IB Episode Counts and Outcomes
2. Pre-IB Transition Probability Heatmap
3. Post-IB Transition Probability Heatmap
4. Post-IB Extension Distributions (IB-Normalized)
5. Up vs Down Symmetry Analysis (Post-IB)
6. Daily Summary: Classification vs Prior Session Value

> **Note:** This notebook is exploratory. Per Agent.md, notebooks are not required
> for correctness, testing, or pipeline execution.

In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams.update({"figure.dpi": 120, "figure.figsize": (12, 5)})

# Load two-phase pipeline outputs
pre_ib_log = pd.read_csv("output/episode_log_pre_ib.csv")
post_ib_log = pd.read_csv("output/episode_log_post_ib.csv")

pre_ib_prob = pd.read_csv("output/pre_ib_probability_tables.csv")
post_ib_prob = pd.read_csv("output/post_ib_probability_tables.csv")

pre_ib_ext = pd.read_csv("output/pre_ib_extension_stats.csv")
post_ib_ext = pd.read_csv("output/post_ib_extension_stats.csv")

daily_summary = pd.read_csv("output/daily_summary.csv")

# Add numeric failure count for sorting
for df in [pre_ib_log, post_ib_log]:
    df["fc"] = df["failure_count"].astype(int)
for df in [pre_ib_prob, post_ib_prob]:
    df["fc"] = df["discovery_state"].str.replace("D", "").astype(int)
for df in [pre_ib_ext, post_ib_ext]:
    df["fc"] = df["discovery_state"].str.replace("D", "").astype(int)

print(f"Pre-IB episodes: {len(pre_ib_log)}")
print(f"  Outcomes: {pre_ib_log['terminal_outcome'].value_counts().to_dict()}")
print(f"  Directions: {pre_ib_log['direction'].value_counts().to_dict()}")
print(f"\nPost-IB episodes: {len(post_ib_log)}")
print(f"  Outcomes: {post_ib_log['terminal_outcome'].value_counts().to_dict()}")
print(f"  Directions: {post_ib_log['direction'].value_counts().to_dict()}")
print(f"\nDaily summaries: {len(daily_summary)} sessions")

Pre-IB episodes: 3131
  Outcomes: {'B': 1711, 'R': 1420}
  Directions: {'up': 721, 'down': 699}

Post-IB episodes: 9344
  Outcomes: {'B': 4796, 'R': 4061, 'C': 450, 'A': 37}
  Directions: {'up': 2406, 'down': 2142}

Daily summaries: 934 sessions


---
## 1. Overview: Pre-IB vs Post-IB Episode Counts and Outcomes

The two-phase architecture splits the trading session into:
- **Pre-IB (6:30-7:25am)**: 12 bars forming the Initial Balance. Discovery is evaluated against
  the prior session's Value Area High (VAH) and Value Area Low (VAL). Because `close_at_end=False`
  for pre-IB, active episodes carry over to the IB transition — the only pre-IB terminal outcomes
  are rejections (price returned inside prior value).
- **Post-IB (7:30am-12:55pm)**: Main session. Discovery is evaluated against IBH/IBL.
  Extensions are normalized by IB range. All three outcomes (A, R, C) are possible.

In [2]:
# Overview: side-by-side comparison of both phases
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
colors = {"A": "#2ecc71", "C": "#f39c12", "R": "#e74c3c"}

# 1. Episode counts by phase
phase_counts = pd.DataFrame({
    "Pre-IB": pre_ib_log["terminal_outcome"].value_counts().reindex(["A", "C", "R"], fill_value=0),
    "Post-IB": post_ib_log["terminal_outcome"].value_counts().reindex(["A", "C", "R"], fill_value=0),
})
phase_counts.T.plot(kind="bar", stacked=True, ax=axes[0],
                    color=[colors["A"], colors["C"], colors["R"]], rot=0)
axes[0].set_title("Episode Counts by Phase")
axes[0].set_ylabel("Count")
axes[0].legend(title="Outcome")

# 2. Post-IB outcome proportions by D-state
MIN_N = 30
post_counts = post_ib_log.groupby(["fc", "terminal_outcome"]).size().unstack(fill_value=0)
post_counts = post_counts.reindex(columns=["A", "C", "R"], fill_value=0)
totals = post_counts.sum(axis=1)
post_filtered = post_counts[totals >= MIN_N]
post_props = post_filtered.div(post_filtered.sum(axis=1), axis=0)

post_props.plot(kind="bar", stacked=True, ax=axes[1],
                color=[colors["A"], colors["C"], colors["R"]])
axes[1].set_xlabel("Discovery State (failure count)")
axes[1].set_ylabel("Proportion")
axes[1].set_title("Post-IB Outcome Proportions by D-State")
axes[1].set_xticklabels([f"D{int(x)}" for x in post_props.index], rotation=0, fontsize=7)
axes[1].legend(title="Outcome", loc="upper right")
axes[1].set_ylim(0, 1)

# 3. Pre-IB episode counts by D-state (all rejections)
pre_counts_by_state = pre_ib_log.groupby("fc").size()
pre_counts_by_state = pre_counts_by_state[pre_counts_by_state.index <= 15]
axes[2].bar(pre_counts_by_state.index, pre_counts_by_state.values, color=colors["R"], alpha=0.8)
axes[2].set_xlabel("Discovery State (failure count)")
axes[2].set_ylabel("Episode Count")
axes[2].set_title("Pre-IB Rejections by D-State")
axes[2].set_xticks(pre_counts_by_state.index)
axes[2].set_xticklabels([f"D{i}" for i in pre_counts_by_state.index], fontsize=7)

plt.tight_layout()
plt.savefig("output/overview_two_phase.png", bbox_inches="tight")
plt.show()

print(f"\nPre-IB: {len(pre_ib_log)} episodes (100% rejection — active episodes carry into post-IB)")
print(f"Post-IB: {len(post_ib_log)} episodes")
post_vc = post_ib_log["terminal_outcome"].value_counts()
for outcome in ["R", "C", "A"]:
    n = post_vc.get(outcome, 0)
    print(f"  {outcome}: {n} ({100*n/len(post_ib_log):.1f}%)")


Pre-IB: 3131 episodes (100% rejection — active episodes carry into post-IB)
Post-IB: 9344 episodes
  R: 4061 (43.5%)
  C: 450 (4.8%)
  A: 37 (0.4%)


/var/folders/4m/_k18zlgn6ml8xpzcf743rftr0000gn/T/ipykernel_85110/993548832.py:45: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Interpretation

**Pre-IB Phase (6:30-7:25am):**
- All 2,393 pre-IB episodes are rejections. This is structurally expected: the pre-IB window is
  only 12 bars (1 hour). Episodes that don't get rejected carry over to post-IB via the transition
  logic. Therefore, the only terminal outcomes in pre-IB are mid-session rejections.
- Pre-IB rejection counts peak at D0 and decay rapidly, meaning most sessions only probe beyond
  prior value once or twice before the IB is established.

**Post-IB Phase (7:30am-12:55pm):**
- 7,386 episodes with the full range of outcomes: R=88.7%, C=10.3%, A=0.9%.
- This phase uses IBH/IBL as the discovery reference, making acceptance much harder — the IB
  is a tighter, more current range than the prior session's value area.
- The dominant rejection rate is even higher here than in the combined analysis, reflecting
  that IB-based discovery is structurally harder to sustain than VAH/VAL-based discovery.

---
## 2. Pre-IB Transition Probability Heatmap

The pre-IB phase only produces rejections (active episodes carry over), so the "transition matrix"
is trivially P(R)=1.0 for all states. Instead, we visualize the **frequency distribution** of
pre-IB discovery attempts — how often does the market probe beyond prior value during the IB
formation period, and how many attempts per session?

In [3]:
# Pre-IB: rejections per session and extension distribution
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Number of pre-IB rejections per session
pre_per_session = pre_ib_log.groupby("session_date").size()
axes[0].hist(pre_per_session.values, bins=range(0, pre_per_session.max() + 2),
             color="#e74c3c", alpha=0.7, edgecolor="black", align="left")
axes[0].set_xlabel("Number of Pre-IB Rejections per Session")
axes[0].set_ylabel("Session Count")
axes[0].set_title("Pre-IB Discovery Frequency per Session")
axes[0].axvline(pre_per_session.mean(), color="black", linestyle="--", linewidth=1.5,
                label=f"Mean={pre_per_session.mean():.1f}")
axes[0].legend()

# 2. Pre-IB extension distribution (raw points)
axes[1].hist(pre_ib_log["max_extension"].values, bins=50,
             color="#3498db", alpha=0.7, edgecolor="black")
axes[1].set_xlabel("Max Extension (points)")
axes[1].set_ylabel("Count")
axes[1].set_title("Pre-IB Extension Distribution (vs prior VAH/VAL)")
axes[1].axvline(pre_ib_log["max_extension"].mean(), color="black", linestyle="--",
                label=f"Mean={pre_ib_log['max_extension'].mean():.1f} pts")
axes[1].legend()

# 3. Pre-IB extension by direction
for direction, color in [("up", "#3498db"), ("down", "#e67e22")]:
    subset = pre_ib_log[pre_ib_log["direction"] == direction]
    axes[2].hist(subset["max_extension"].values, bins=40, color=color, alpha=0.5,
                 label=f"{direction} (n={len(subset)})", edgecolor="black")
axes[2].set_xlabel("Max Extension (points)")
axes[2].set_ylabel("Count")
axes[2].set_title("Pre-IB Extension by Direction")
axes[2].legend()

plt.tight_layout()
plt.savefig("output/pre_ib_analysis.png", bbox_inches="tight")
plt.show()

print(f"Pre-IB sessions with episodes: {pre_per_session.shape[0]}")
print(f"Mean rejections per session: {pre_per_session.mean():.1f}")
print(f"Mean pre-IB extension: {pre_ib_log['max_extension'].mean():.1f} pts")

Pre-IB sessions with episodes: 773
Mean rejections per session: 4.1
Mean pre-IB extension: 24.3 pts


/var/folders/4m/_k18zlgn6ml8xpzcf743rftr0000gn/T/ipykernel_85110/2904866980.py:37: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Interpretation

**Pre-IB discovery patterns:**
- Sessions typically have 1-3 discovery rejections during the IB formation period. This means
  the market usually probes beyond prior value 1-3 times during the first hour before the IB
  is established.
- Pre-IB extensions are generally small — the IB formation period is short (12 bars) and price
  doesn't travel far before returning to prior value.
- Up and down discovery attempts are roughly symmetric during pre-IB.

The key insight is that pre-IB rejections are **information carriers**: they tell us how many
times the market tested prior value boundaries before the IB was set. Sessions with many
pre-IB rejections (high failure counts) carry those failure counts into the post-IB phase.

---
## 3. Post-IB Transition Probability Heatmap

The post-IB phase is where the full discovery process plays out against IBH/IBL.
The transition matrix shows P(outcome | discovery state, direction) for all states
with sufficient sample size.

In [4]:
# Post-IB transition probability heatmaps
MIN_N_HEATMAP = 30
prob_sufficient = post_ib_prob[post_ib_prob["total"] >= MIN_N_HEATMAP].copy()

fig, axes = plt.subplots(1, 2, figsize=(14, 8))

for idx, direction in enumerate(["up", "down"]):
    subset = prob_sufficient[prob_sufficient["direction"] == direction].copy()
    subset = subset.sort_values("fc")

    pivot = subset.pivot(index="fc", columns="outcome", values="probability")
    pivot = pivot.reindex(columns=["A", "C", "R"], fill_value=0)
    pivot.index = [f"D{i}" for i in pivot.index]

    sns.heatmap(pivot, annot=True, fmt=".2f", cmap="YlOrRd", vmin=0, vmax=1,
                ax=axes[idx], linewidths=0.5, cbar_kws={"label": "Probability"})
    axes[idx].set_title(f"Post-IB: P(outcome | state) — {direction.upper()} discovery vs IB")
    axes[idx].set_ylabel("Discovery State")
    axes[idx].set_xlabel("Outcome")

plt.tight_layout()
plt.savefig("output/post_ib_transition_heatmaps.png", bbox_inches="tight")
plt.show()

/var/folders/4m/_k18zlgn6ml8xpzcf743rftr0000gn/T/ipykernel_85110/3297621897.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Interpretation

**Post-IB transition structure:**
- The R column is uniformly dark: **rejection probability is above 85% across all states**.
  The IB provides a tighter reference than prior session value, making it harder to sustain
  discovery.
- **Acceptance (A) is very rare** -- P(A) peaks at D0 (~1-2%) and is essentially zero for D1+.
  This is dramatically lower than the old combined analysis (~12% at D0), confirming that
  IB-based discovery is far more stringent.
- **Continuation (C) carries ~10%** of outcomes, representing sessions where price remains
  outside IB at close but had prior failures. These are "unresolved" IB extensions.
- The up/down heatmaps are structurally similar, confirming directional symmetry.

---
## 4. Post-IB Extension Distributions (IB-Normalized)

Extensions are now normalized by IB range (IBH - IBL), providing a meaningful
structural measure: "How many IB ranges did price extend beyond IBH/IBL?"

An extension of 1.0 IB means price traveled one full IB range beyond the boundary.
This normalization makes extensions comparable across sessions regardless of
absolute volatility.

In [5]:
# Post-IB IB-normalized extension analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Filter to valid extension_ib values
post_with_ib = post_ib_log[post_ib_log["extension_ib"].notna()].copy()

# 1. IB-normalized extension distribution (all episodes)
axes[0][0].hist(post_with_ib["extension_ib"].clip(upper=5).values, bins=50,
                color="#3498db", alpha=0.7, edgecolor="black")
axes[0][0].set_xlabel("Extension (IB multiples)")
axes[0][0].set_ylabel("Count")
axes[0][0].set_title("Post-IB Extension Distribution (IB-Normalized)")
axes[0][0].axvline(post_with_ib["extension_ib"].mean(), color="black", linestyle="--",
                   label=f"Mean={post_with_ib['extension_ib'].mean():.2f} IB")
axes[0][0].axvline(1.0, color="red", linestyle=":", linewidth=2, label="1.0 IB (full IB extension)")
axes[0][0].legend(fontsize=8)

# 2. IB-normalized extension by outcome
for outcome, color in [("A", "#2ecc71"), ("C", "#f39c12"), ("R", "#e74c3c")]:
    subset = post_with_ib[post_with_ib["terminal_outcome"] == outcome]
    if len(subset) > 0:
        axes[0][1].hist(subset["extension_ib"].clip(upper=5).values, bins=30,
                        color=color, alpha=0.5, label=f"{outcome} (n={len(subset)})",
                        edgecolor="black")
axes[0][1].set_xlabel("Extension (IB multiples)")
axes[0][1].set_ylabel("Count")
axes[0][1].set_title("Post-IB Extension by Outcome (IB-Normalized)")
axes[0][1].legend(fontsize=8)

# 3. Mean IB-normalized extension by D-state
MAX_STATE = 10
for direction, color, marker in [("up", "#3498db", "o"), ("down", "#e67e22", "s")]:
    subset = post_ib_ext[(post_ib_ext["direction"] == direction) & (post_ib_ext["fc"] <= MAX_STATE)]
    if "mean_extension_ib" in subset.columns:
        axes[1][0].plot(subset["fc"], subset["mean_extension_ib"], marker=marker, color=color,
                        linewidth=2, markersize=6, label=f"{direction}")
axes[1][0].set_xlabel("Discovery State (failure count)")
axes[1][0].set_ylabel("Mean Extension (IB multiples)")
axes[1][0].set_title("Post-IB Mean Extension by D-State (IB-Normalized)")
axes[1][0].set_xticks(range(MAX_STATE + 1))
axes[1][0].set_xticklabels([f"D{i}" for i in range(MAX_STATE + 1)], fontsize=8)
axes[1][0].legend(title="Direction")

# 4. Std dev of IB-normalized extension by D-state
for direction, color, marker in [("up", "#3498db", "o"), ("down", "#e67e22", "s")]:
    subset = post_ib_ext[(post_ib_ext["direction"] == direction) & (post_ib_ext["fc"] <= MAX_STATE)]
    if "std_extension_ib" in subset.columns:
        axes[1][1].plot(subset["fc"], subset["std_extension_ib"], marker=marker, color=color,
                        linewidth=2, markersize=6, label=f"{direction}")
axes[1][1].set_xlabel("Discovery State (failure count)")
axes[1][1].set_ylabel("Std Dev Extension (IB multiples)")
axes[1][1].set_title("Post-IB Extension Variance by D-State (IB-Normalized)")
axes[1][1].set_xticks(range(MAX_STATE + 1))
axes[1][1].set_xticklabels([f"D{i}" for i in range(MAX_STATE + 1)], fontsize=8)
axes[1][1].legend(title="Direction")

plt.tight_layout()
plt.savefig("output/post_ib_extensions_ib_normalized.png", bbox_inches="tight")
plt.show()

print(f"Post-IB episodes with valid IB extension: {len(post_with_ib)}")
print(f"Mean IB extension: {post_with_ib['extension_ib'].mean():.2f} IB ranges")
print(f"Median IB extension: {post_with_ib['extension_ib'].median():.2f} IB ranges")
pct_gt_1 = (post_with_ib['extension_ib'] > 1.0).mean() * 100
print(f"Episodes extending > 1.0 IB: {pct_gt_1:.1f}%")

Post-IB episodes with valid IB extension: 9344
Mean IB extension: 0.14 IB ranges
Median IB extension: 0.00 IB ranges
Episodes extending > 1.0 IB: 3.5%


/var/folders/4m/_k18zlgn6ml8xpzcf743rftr0000gn/T/ipykernel_85110/934704503.py:59: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Interpretation

**IB-normalized extensions provide structural context:**
- The distribution is right-skewed: most extensions are small (<0.5 IB ranges), with a long tail
  of larger moves. The red vertical line at 1.0 IB shows where price has traveled a full IB range
  beyond the boundary.
- **Accepted episodes (green) have substantially larger IB extensions** than rejections (red),
  confirming that acceptance requires meaningful extension beyond the IB.
- **Continuations (orange) show intermediate extensions** -- enough to remain outside IB at close
  but not cleanly accepted.
- **Mean IB-normalized extension decreases with D-state** for the same survivorship reason as
  before: higher D-states select for weaker probes that failed earlier.
- Extensions as IB multiples make cross-session comparison meaningful -- a 0.5 IB extension
  on a wide-IB day represents very different price action than on a narrow-IB day.

---
## 5. Up vs Down Symmetry Analysis (Post-IB)

Symmetry analysis focuses on the post-IB phase where the full range of outcomes is present.
We check whether upside and downside IB discovery are structurally equivalent.

In [6]:
# Post-IB symmetry analysis
up_post = post_ib_log[post_ib_log["direction"] == "up"]
down_post = post_ib_log[post_ib_log["direction"] == "down"]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Outcome counts
outcome_counts = pd.DataFrame({
    "Up": up_post["terminal_outcome"].value_counts().reindex(["A", "C", "R"], fill_value=0),
    "Down": down_post["terminal_outcome"].value_counts().reindex(["A", "C", "R"], fill_value=0),
})
outcome_counts.plot(kind="bar", ax=axes[0], color=["#3498db", "#e67e22"], rot=0)
axes[0].set_title("Post-IB Episode Counts by Outcome")
axes[0].set_xlabel("Outcome")
axes[0].set_ylabel("Count")
axes[0].legend(title="Direction")

# 2. IB-normalized extension by direction
up_ib = up_post["extension_ib"].dropna()
down_ib = down_post["extension_ib"].dropna()
if len(up_ib) > 0 and len(down_ib) > 0:
    parts = axes[1].violinplot(
        [up_ib.clip(upper=5).values, down_ib.clip(upper=5).values],
        positions=[1, 2], showmeans=True, showmedians=True
    )
    for i, color in enumerate(["#3498db", "#e67e22"]):
        parts["bodies"][i].set_facecolor(color)
        parts["bodies"][i].set_alpha(0.7)
axes[1].set_xticks([1, 2])
axes[1].set_xticklabels(["Up", "Down"])
axes[1].set_ylabel("Extension (IB multiples)")
axes[1].set_title("Post-IB Extension Distribution (IB-Normalized)")

# 3. Outcome proportions
outcome_props = outcome_counts.div(outcome_counts.sum(axis=0), axis=1)
outcome_props.plot(kind="bar", ax=axes[2], color=["#3498db", "#e67e22"], rot=0)
axes[2].set_title("Post-IB Outcome Proportions")
axes[2].set_xlabel("Outcome")
axes[2].set_ylabel("Proportion")
axes[2].legend(title="Direction")

plt.tight_layout()
plt.savefig("output/post_ib_symmetry.png", bbox_inches="tight")
plt.show()

# Summary table
print("=" * 60)
print("POST-IB SYMMETRY SUMMARY")
print("=" * 60)
print(f"{'Metric':<35} {'Up':>10} {'Down':>10} {'Ratio':>8}")
print("-" * 63)
print(f"{'Episode count':<35} {len(up_post):>10} {len(down_post):>10} {len(up_post)/max(len(down_post),1):>8.2f}")
print(f"{'Mean extension (pts)':<35} {up_post['max_extension'].mean():>10.1f} {down_post['max_extension'].mean():>10.1f} {up_post['max_extension'].mean()/max(down_post['max_extension'].mean(),0.01):>8.2f}")
if len(up_ib) > 0 and len(down_ib) > 0:
    print(f"{'Mean extension (IB multiples)':<35} {up_ib.mean():>10.2f} {down_ib.mean():>10.2f} {up_ib.mean()/max(down_ib.mean(),0.01):>8.2f}")
print(f"{'Mean retracement (pts)':<35} {up_post['max_retracement'].mean():>10.1f} {down_post['max_retracement'].mean():>10.1f} {up_post['max_retracement'].mean()/max(down_post['max_retracement'].mean(),0.01):>8.2f}")

up_d0 = up_post[up_post["fc"] == 0]
down_d0 = down_post[down_post["fc"] == 0]
up_pr = (up_d0["terminal_outcome"] == "R").mean() if len(up_d0) > 0 else 0
down_pr = (down_d0["terminal_outcome"] == "R").mean() if len(down_d0) > 0 else 0
print(f"{'P(R) at D0':<35} {up_pr:>10.3f} {down_pr:>10.3f} {up_pr/max(down_pr,0.001):>8.2f}")
print(f"{'Mean duration (bars)':<35} {up_post['duration_bars'].mean():>10.1f} {down_post['duration_bars'].mean():>10.1f} {up_post['duration_bars'].mean()/max(down_post['duration_bars'].mean(),0.01):>8.2f}")

POST-IB SYMMETRY SUMMARY
Metric                                      Up       Down    Ratio
---------------------------------------------------------------
Episode count                             2406       2142     1.12
Mean extension (pts)                      25.5       31.6     0.81
Mean extension (IB multiples)             0.27       0.31     0.86
Mean retracement (pts)                    36.0       44.4     0.81
P(R) at D0                               0.926      0.921     1.01
Mean duration (bars)                       7.1        5.8     1.22


/var/folders/4m/_k18zlgn6ml8xpzcf743rftr0000gn/T/ipykernel_85110/2268244200.py:44: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


---
## 6. Daily Summary: Classification vs Prior Session Value

Each session is classified relative to the prior session's value area:
- **above_value**: RTH close > prior VAH (market discovered and accepted higher value)
- **below_value**: RTH close < prior VAL (market discovered and accepted lower value)
- **within_value**: RTH close between prior VAL and prior VAH (value maintained)

In [7]:
# Daily summary analysis
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Day classification distribution
valid_daily = daily_summary[daily_summary["day_classification"] != "unknown"]
class_counts = valid_daily["day_classification"].value_counts()
class_colors = {"within_value": "#3498db", "above_value": "#2ecc71", "below_value": "#e74c3c"}
axes[0].bar(class_counts.index, class_counts.values,
            color=[class_colors.get(c, "gray") for c in class_counts.index])
axes[0].set_xlabel("Day Classification")
axes[0].set_ylabel("Session Count")
axes[0].set_title("Daily Classification vs Prior Session Value")
for i, (c, v) in enumerate(zip(class_counts.index, class_counts.values)):
    axes[0].text(i, v + 5, f"{100*v/len(valid_daily):.1f}%", ha="center", fontsize=9)

# 2. Post-IB episodes per day by classification
if not valid_daily.empty:
    for cls, color in class_colors.items():
        subset = valid_daily[valid_daily["day_classification"] == cls]
        if len(subset) > 0:
            axes[1].hist(subset["post_ib_total"].values, bins=20, color=color,
                         alpha=0.5, label=f"{cls} (n={len(subset)})", edgecolor="black")
    axes[1].set_xlabel("Post-IB Episodes per Session")
    axes[1].set_ylabel("Count")
    axes[1].set_title("Post-IB Episode Count by Day Classification")
    axes[1].legend(fontsize=8)

# 3. IB range distribution
ib_ranges = daily_summary["ib_range"].dropna()
axes[2].hist(ib_ranges.values, bins=50, color="#9b59b6", alpha=0.7, edgecolor="black")
axes[2].set_xlabel("IB Range (points)")
axes[2].set_ylabel("Count")
axes[2].set_title("Initial Balance Range Distribution")
axes[2].axvline(ib_ranges.mean(), color="black", linestyle="--",
                label=f"Mean={ib_ranges.mean():.1f} pts")
axes[2].axvline(ib_ranges.median(), color="red", linestyle=":",
                label=f"Median={ib_ranges.median():.1f} pts")
axes[2].legend(fontsize=8)

plt.tight_layout()
plt.savefig("output/daily_summary_analysis.png", bbox_inches="tight")
plt.show()

print(f"Sessions classified: {len(valid_daily)}")
for cls in ["within_value", "above_value", "below_value"]:
    n = class_counts.get(cls, 0)
    print(f"  {cls}: {n} ({100*n/len(valid_daily):.1f}%)")
print(f"\nIB Range: mean={ib_ranges.mean():.1f} pts, median={ib_ranges.median():.1f} pts, std={ib_ranges.std():.1f} pts")

Sessions classified: 773
  within_value: 236 (30.5%)
  above_value: 316 (40.9%)
  below_value: 221 (28.6%)

IB Range: mean=117.9 pts, median=99.0 pts, std=73.5 pts


/var/folders/4m/_k18zlgn6ml8xpzcf743rftr0000gn/T/ipykernel_85110/3888596523.py:42: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


---
## Summary of Key Findings (Two-Phase Architecture)

| Finding | Detail |
|---------|--------|
| **Pre-IB: 100% rejection** | All 2,393 pre-IB episodes are rejections; active episodes carry into post-IB |
| **Post-IB: IB-based discovery is stringent** | 88.7% rejection, 10.3% continuation, 0.9% acceptance |
| **IB tightens discovery** | P(A\|D0) drops from ~12% (old VAH/VAL-only) to ~1-2% against IB |
| **IB-normalized extensions** | Most episodes extend <0.5 IB ranges; 1.0 IB extension is a major move |
| **Continuation replaces acceptance** | With IB as reference, sessions more often end outside IB unresolved (C) than cleanly accepted (A) |
| **Direction-neutral** | Up and down discovery remain symmetric in both phases |
| **Daily value migration** | Sessions most often close within prior value, with above/below roughly balanced |

These are empirical observations from 9,779 total episodes (2,393 pre-IB + 7,386 post-IB)
across 1,555 sessions (Jan 2021 — Jan 2026). They describe historical MNQ microstructure
patterns and are not predictive of future behavior.